In [28]:
import cx_Oracle
import os
import io

In [29]:
os.environ["NLS_LANG"] = "POLISH_POLAND.AL32UTF8"
connection = cx_Oracle.connect("C##PPAWLUK", "#Lex2018", "144.21.89.60:1521/ORCL.591263988.oraclecloud.internal")

In [30]:
cur = connection.cursor()

In [31]:
cur.execute('DELETE FROM "C##PPAWLUK"."DOKUMENT"')
cur.execute('commit')

In [32]:
nameDocuments = os.listdir("dokumenty")

In [33]:
def clearTxt(myfile):
    myfile = myfile.replace('\n', ' ').replace(',', '').replace('.', '').replace('!', '').replace('?', '').replace('/',
                                                                                                                   '').replace(
        '\\', '').replace(';', '').replace('"', ' ').replace('\'', ' ').replace(']', '').replace('[', '').replace(')',
                                                                                                                  '').replace(
        '(',
        '').replace(
        ':', '').replace('`', '').replace('@', '').replace('#', '').replace('$', '').replace('%', '').replace(
        '^', '').replace('&', '').replace('*', '').replace('-', ' ').replace('_', ' ').replace('+', ' ').replace('=',
                                                                                                                 ' ').replace(
        '0', '').replace('1', '').replace('2', '').replace('3', '').replace('4', '').replace('5', '').replace('6',
                                                                                                              '').replace(
        '7', '').replace('8', '').replace('9', '').replace('>', ' ').replace('<', ' ').replace('{', '').replace('}',
                                                                                                                '').replace(
        '§', '').replace('\', '').replace('\\ufeff', '').replace(' ', ' ').replace('–', ' ').replace('“', '').replace(
        '„', '').replace('…', '').replace('	', '').replace('’', ' ').replace('”', '').replace('  ', ' ').replace(
        '  ', ' ')
    return myfile

In [34]:
def stopList(myfile):
    replaceFile = myfile
    for word in myfile:
        stopword = cur.execute(
            'select STWORD from STOPWORDS where STWORD = \'{}\' and rownum=1'.format(
                word))
        for i in stopword:
            replaceFile.remove(i[0])
    removeTab = {'	', '\n', ''}
    for remove in removeTab:
        while (remove in replaceFile):
            replaceFile.remove(remove)
    return replaceFile

In [35]:
def slowikPolskiego(replaceFile):
    j = 0
    for word in replaceFile:
        dictionary = cur.execute(
            'select slow_forma_podstawowa from slownik where slow_wyraz = \'{}\' and rownum=1'.format(
                word))
        for i in dictionary:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile


In [36]:
def slowikSynonimow1(replaceFile):
    j = 0
    for word in replaceFile:
        thesaurus = cur.execute(
            'select slow_forma_podstawowa from slownik_synonimow where slow_wyraz = \'{}\' and slow_wyraz != slow_forma_podstawowa and rownum=1 '.format(
                word))
        for i in thesaurus:
            replaceFile.__setitem__(j, i[0])
            j = j + 1
    return replaceFile

In [37]:
def pdfparser(data, nr):
    file = io.open(data, mode="r", encoding='utf-8-sig')
    myfile = file.read()
    myfile = myfile.lower()
    myfile = clearTxt(myfile)
    myfile = myfile.split(' ')
    replaceFile = stopList(myfile)
    replaceFile = slowikPolskiego(replaceFile)
    replaceFile = slowikSynonimow1(replaceFile)
    txt = ''
    for i in range(0, len(replaceFile), 1):
        txt = txt + ' ' + (replaceFile.__getitem__(i))
        if i % 200 == 0 and i != 0:
            cur.execute(
                'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                    nameDocuments[nr], txt))
            cur.execute('commit')
            txt = ''
    if (txt != ''):
        cur.execute(
            'INSERT INTO "C##PPAWLUK"."DOKUMENT" (NAZWA_DOKUMENTU, TEKST) VALUES( \'{}\', \'{}\')'.format(
                nameDocuments[nr], txt))
        cur.execute('commit')

In [38]:
for i in range(0, len(nameDocuments), 1):
    pdfparser("D:\BSpy\!materiały na studia\Praca inżynierska\Implementacja\PorownywanieDokumentow\dokumenty\{}".format(
        nameDocuments[i]), i)

In [39]:
cur.close()

In [40]:
connection.close()